# Modeling Naive Bayes

In [2]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd

/Users/roberthillery/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
df = pd.read_csv('csv_files/9_1_all_books_df.csv')

In [4]:
# load in database

In [5]:
df.columns

Index(['author', 'description', 'format', 'genre', 'month', 'pages',
       'publisher', 'rating', 'title', 'year', 'best_seller'],
      dtype='object')

In [6]:
df.shape

(1373, 11)

In [7]:
df.head()

,author,description,format,genre,month,pages,publisher,rating,title,year,best_seller
0,Liane Moriarty,Six responsible adults. Three cute kids. One s...,Hardcover,Fiction,July,415,Flatiron Books,3.56,Truly Madly Guilty,2016.0,0
1,Bryn Greenwood,"As the daughter of a meth dealer, Wavy knows n...",Hardcover,Fiction,August,346,Thomas Dunne Books,4.07,All the Ugly and Wonderful Things,2016.0,0
2,Emma Cline,"Northern California, during the violent end of...",Hardcover,Fiction,June,355,Random House,3.47,The Girls,2016.0,0
3,Cynthia D'Aprix Sweeney,"<i>A warm, funny and acutely perceptive debut ...",Hardcover,Fiction,March,368,Ecco,3.42,The Nest,2016.0,0
4,Jodi Picoult,Ruth Jefferson is a labor and delivery nurse a...,Hardcover,Fiction,October,480,Ballantine Books,4.35,Small Great Things,2016.0,0


In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(naive_df['description'], naive_df['best_seller'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)